In [2]:
####スパース多項ロジットモデル####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

#np.random.seed(8742)   #シードを設定

In [10]:
####データの発生####
##データの設定
hh1 = 5000   #学習用データのサンプル数
hh2 = 3000   #検証用データのサンプル数
hh = hh1 + hh2   #総サンプル数
select = 10   #選択肢数
k1 = 300   #連続変数の説明変数
k2 = 200   #離散変数の説明変数
k = k1 + k2   #説明変数数


array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [176]:
##説明変数の生成
#連続変数の生成
X1 = numpy.random.multivariate_normal(np.repeat(0, k1), np.identity(k1), hh)

#離散変数の生成
X2 = np.zeros((hh, k2))
for j in range(k2):
    pr = uniform(0.25, 0.75, 1)
    X2[:, j] = numpy.random.binomial(1, pr, hh)
    
#データの結合
intercept = np.repeat(1, hh)[:, np.newaxis]
X = np.concatenate((intercept, X1, X2), axis=1)
X1 = X[0:hh1, :]
X2 = X[hh1:hh, :]

In [177]:
##応答変数の生成
#パラメータの設定
zeros = np.repeat(0, k+1)[:, np.newaxis]
theta = np.concatenate((np.random.multivariate_normal(np.repeat(0, k+1), 0.05 * np.identity(k+1), select-1).T, zeros), axis=1)
theta[np.abs(theta0) > 0.2] = 0   #パラメータをゼロにシュリンク
thetat = theta

#ロジットと選択確率を設定
logit = np.dot(X, theta)   #ロジット
Pr = np.exp(logit) / np.matlib.repmat(np.sum(np.exp(logit), axis=1), select, 1).T   #選択確率

#多項分布から選択結果を生成
y = np.zeros((hh, select))
for i in range(hh):
   y[i, :] = np.random.multinomial(1, Pr[i, ], 1)
y1 = y[0:hh1, :]
y2 = y[hh1:hh, :]
np.sum(y, axis=0)

array([  929.,   557.,   953.,  1251.,   525.,   607.,   978.,  1413.,
         620.,   167.])

In [178]:
####総当たり座標降下法でスパース多項ロジットモデルを推定####
##多項ロジットモデルの対数尤度の関数
def define_likelihood(y, X, theta, select):
    
    #ロジットと選択確率の計算
    logit = np.dot(X, theta)
    Pr = np.exp(logit) / np.matlib.repmat(np.sum(np.exp(logit), axis=1), select, 1).T

    #対数尤度の計算
    LL = np.sum(y * np.log(Pr))
    return(LL)

##座標降下法の設定
#正則化パラメータの設定
n = 15
lambda_vec = np.arange(0.001, 0.01, (0.01-0.001)/n)
n_lambda = hh1 * lambda_vec
X1_sq = np.power(X1, 2)

#ベストな対数尤度
LLbest = define_likelihood(y2, X2, thetat, select)

#パラメータの格納用配列
LLtest = np.array(())
THETA = np.zeros((k+1, select, n))

In [195]:
####スパース正則化多項ロジットモデルで正則化パラメータを最適化####
##アルゴリズムの設定
#パラメータの初期値
theta = np.zeros((k+1, select))
LL = define_likelihood(y1, X1, thetat, select)

#アルゴリズムの更新ステータス
LLs = LL
iter = 1
dl = -100   #対数尤度の差の初期値
tol = 2.5   #収束判定
LL1 = LL   #対数尤度の初期値


##ロジットと選択確率を定義
logit = np.dot(X1, theta)
Pr = np.exp(logit) / np.matlib.repmat(np.sum(np.exp(logit), axis=1), select, 1).T

##パラメータの更新
#微分係数のパラメータを更新
w = (Pr * (1 - Pr))[:, -select]
z = logit[:, -select] + (y1 - Pr)[:, -select] / w

#切片の更新
theta[1, -select] 
np.sum(w * (z - logit[:, -select]), axis=0) / np.sum(w, axis=0)

0.14666666666666661

In [196]:
w * (z - logit[:, -select])

array([ 0.9, -0.1, -0.1, ..., -0.1, -0.1, -0.1])

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])